In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import os
import random
import re
import torch
from datetime import datetime


def dataset(path, transform):

  samples = []
  root_folder = path
  folders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

  for folder in folders:
      root_folder = f'/content/drive/MyDrive/HomeworkDataset/{folder}/images'
      images = [f for f in os.listdir(root_folder) if f.lower().endswith(('.tif', '.tiff'))]
      random.shuffle(images)
      image_pairs = [(images[i], images[i + 1]) for i in range(0, len(images), 2) if i + 1 < len(images)]

      for image_pair in image_pairs:
        pattern = r'(\d{4})_(\d{2})'
        date1 = re.search(pattern, image_pair[0])
        date2 = re.search(pattern, image_pair[1])
        d1 = datetime.strptime(date1[0], "%Y_%m")
        d2 = datetime.strptime(date2[0], "%Y_%m")
        delta = abs((d1.year - d2.year) * 12 + d1.month - d2.month)
        samples.append((image_pair[0],image_pair[1],delta))

  return samples



In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#I tried to predict the time_skip, this is why I tried to use Siamese network architecture
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 10),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 7),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 128, 4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 4),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.Sigmoid()
        )

    def forward_one(self, x):
        x = self.cnn(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

class SiameseDataset(Dataset):
    def __init__(self, image_pairs, labels, transform=None):
        self.image_pairs = image_pairs
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_pairs)

    def __getitem__(self, idx):
        img1, img2 = self.image_pairs[idx]
        label = self.labels[idx][2]

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, label

def train_siamese_network(siamese_model, dataloader, criterion, optimizer):
    siamese_model.train()
    total_loss = 0.0
    for batch in dataloader:
        input1, input2, labels = batch
        optimizer.zero_grad()
        output1, output2 = siamese_model(input1, input2)
        loss = criterion(output1, output2, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def validate_siamese_network(siamese_model, dataloader, criterion):
    siamese_model.eval()
    total_loss = 0.0
    correct = 0
    with torch.no_grad():
        for batch in dataloader:
            input1, input2, labels = batch
            output1, output2 = siamese_model(input1, input2)
            loss = criterion(output1, output2, labels)
            total_loss += loss.item()
            predicted = (output1 - output2).sign()
            correct += (predicted == labels).sum().item()
    return total_loss / len(dataloader), correct / len(dataloader.dataset)

siamese_model = SiameseNetwork()
optimizer = optim.Adam(siamese_model.parameters(), lr=0.00006)
criterion = nn.BCEWithLogitsLoss()

samples=dataset('/content/drive/MyDrive/HomeworkDataset',0)
print(samples[0][1])
training,validation,test=torch.utils.data.random_split(samples, [0.7,0.15,0.15])
image_pairs = [(image1, image2) for image1, image2, _ in training]
timeDiff = [timeDiff for _, _, timeDiff in training]
siamese_dataset = SiameseDataset(image_pairs, training)
dataloader = DataLoader(siamese_dataset, batch_size=12, shuffle=True)

def run(siamese_model, dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        train_loss = train_siamese_network(siamese_model, dataloader, criterion, optimizer)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")

num_epochs = 10
run(siamese_model, dataloader, criterion, optimizer, num_epochs)


run(training, validation, 2)

global_monthly_2020_01_mosaic_L15-1848E-0793N_7394_5018_13.tif


TypeError: ignored